# Start

In [1]:
# Librerías

import numpy as np
import pandas as pd
import sys

# Funciones del archivo fun.py

sys.path.append('/home/marta/Desktop/IH/Lets_blockbust/SCC')
from fun import firstglance

In [2]:
# Importamos los datos de todas las tablas disponibles

actor = pd.read_csv('../data/actor.csv')
category = pd.read_csv('../data/category.csv')
film = pd.read_csv('../data/film.csv')
language = pd.read_csv('../data/language.csv') 
actorfilm = pd.read_csv('../data/old_HDD.csv') # Esta conectará actores y películas, le cambiamos el nombre
rental = pd.read_csv('../data/rental.csv') 
inventory = pd.read_csv('../data/inventory.csv')

# Análisis preliminar y limpieza básica

Vamos a echar un ojo a cada tabla, con sus nulos, la información general y su tamaño. 
Para ello diseñamos una función llamada firstglance(x) 
que guardaremos en fun.py dentro de SCC y usaremos a partir de ahora.

# Tabla actor

In [3]:
firstglance(actor)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33



Shape: (200, 4)

Null values count:

actor_id       0
first_name     0
last_name      0
last_update    0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


None

In [4]:
# Es una tabla sin nulos, con la información aparentemente bien organizada según el head.
# Vamos columna a columna a ver si los datos son homogéneos.

actor.actor_id # El id está bien, hay 200 datos y están bien etiquetados.

0        1
1        2
2        3
3        4
4        5
      ... 
195    196
196    197
197    198
198    199
199    200
Name: actor_id, Length: 200, dtype: int64

In [5]:
actor.first_name.value_counts() # Todo bien parece. Hay varios actores y actrices con el mismo nombre

PENELOPE    4
JULIA       4
KENNETH     4
GENE        3
GROUCHO     3
           ..
HENRY       1
DUSTIN      1
JUDE        1
CARMEN      1
THORA       1
Name: first_name, Length: 128, dtype: int64

In [6]:
actor.last_name.value_counts() # Lo mismo.

KILMER     5
TEMPLE     4
NOLTE      4
GUINESS    3
ALLEN      3
          ..
JOLIE      1
DAMON      1
CRUZ       1
SINATRA    1
WALKEN     1
Name: last_name, Length: 121, dtype: int64

In [7]:
# Podría ser interesante unir estas dos últimas columnas. Hagámoslo.

actor['name'] = actor.apply(lambda x:'%s_%s' % (x['first_name'],x['last_name']), axis = 1)

# Ahora nos deshacemos de las dos antiguas:

actor.drop(columns = ['first_name','last_name'], inplace = True)

actor.head()

,actor_id,last_update,name
0,1,2006-02-15 04:34:33,PENELOPE_GUINESS
1,2,2006-02-15 04:34:33,NICK_WAHLBERG
2,3,2006-02-15 04:34:33,ED_CHASE
3,4,2006-02-15 04:34:33,JENNIFER_DAVIS
4,5,2006-02-15 04:34:33,JOHNNY_LOLLOBRIGIDA


In [8]:
actor.name.value_counts() 

SUSAN_DAVIS             2
EWAN_GOODING            1
DARYL_CRAWFORD          1
GRETA_KEITEL            1
JANE_JACKMAN            1
                       ..
MICHELLE_MCCONAUGHEY    1
ADAM_GRANT              1
SEAN_WILLIAMS           1
GARY_PENN               1
THORA_TEMPLE            1
Name: name, Length: 199, dtype: int64

In [9]:
# Fíjate, hay dos Susan Davis con ids distintos! Habrá que añadir un tercer apellido quizá. 
# Como no podemos buscarla en internet (no son actores reales), eliminamos una Susan y reseteamos 
# el índice y el actor_id:

actor.drop_duplicates(subset = 'name', keep = 'first', inplace = True, ignore_index = True)
actor.reset_index(inplace = True, drop = True)
actor.actor_id = [i for i in range(1,actor.shape[0]+1)]
actor.name.value_counts()  # Hecho

PENELOPE_GUINESS        1
KEVIN_GARLAND           1
DARYL_CRAWFORD          1
GRETA_KEITEL            1
JANE_JACKMAN            1
                       ..
MICHELLE_MCCONAUGHEY    1
ADAM_GRANT              1
SEAN_WILLIAMS           1
GARY_PENN               1
THORA_TEMPLE            1
Name: name, Length: 199, dtype: int64

In [10]:
actor.head()

,actor_id,last_update,name
0,1,2006-02-15 04:34:33,PENELOPE_GUINESS
1,2,2006-02-15 04:34:33,NICK_WAHLBERG
2,3,2006-02-15 04:34:33,ED_CHASE
3,4,2006-02-15 04:34:33,JENNIFER_DAVIS
4,5,2006-02-15 04:34:33,JOHNNY_LOLLOBRIGIDA


In [11]:
# Vamos a por la columna last_update:

actor.last_update.value_counts()

2006-02-15 04:34:33    199
Name: last_update, dtype: int64

In [12]:
# Qué interesante... TODAS las entradas de last_update en la tabla de actores valen lo mismo. 
# Esta columna sobra, es mucho espacio para una misma información.

# Igual tendría sentido crear una tabla nueva con el update de cada una de las demás, 
# si pasa lo mismo en todas con distintas fechas. Metámosla en un nuevo dataframe 
# y quitemos la columna de aquí que estorba:

a = actor.last_update[0]

updates = pd.DataFrame(data = {'table': ['actor'],'last_update':[a]})
actor.drop(columns = 'last_update', inplace = True)

actor.head()

,actor_id,name
0,1,PENELOPE_GUINESS
1,2,NICK_WAHLBERG
2,3,ED_CHASE
3,4,JENNIFER_DAVIS
4,5,JOHNNY_LOLLOBRIGIDA


# Tabla film

In [13]:
firstglance(film)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42



Shape: (1000, 13)

Null values count:

film_id                    0
title                      0
description                0
release_year               0
language_id                0
original_language_id    1000
rental_duration            0
rental_rate                0
length                     0
replacement_cost           0
rating                     0
special_features           0
last_update                0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null  

None

Vemos que hay una columna que es toda de nulos, el original_language_id. No la vamos a borrar de momento, porque a lo mejor podemos relacionarla como foreign key con alguna de las columnas de la tabla 'language' que hay por ahí. Por ahora le asignamos el proxy 7 para saber que es un campo desconocido (añadiremos idioma 'Unknown' a la tabla de language luego, y le asignaremos id 7).

In [14]:
film.original_language_id = 7 

In [15]:
# Vamos a comprobar si pasa lo mismo en last_update que en la tabla anterior, 
# y si ocurre procederemos de la misma manera.

film.last_update.value_counts()

2006-02-15 05:03:42    1000
Name: last_update, dtype: int64

In [16]:
# Es el mismo valor para toda la tabla (que no el mismo que en la columna actor). 
# Lo guardamos en el dataframe de updates y borramos la columna:

a = film.last_update[0]
updates = pd.concat([updates, pd.DataFrame({'table': 'film','last_update':[a]})], axis = 0)

film.drop(columns = 'last_update', inplace = True)

film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,7,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,7,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,7,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes"
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,7,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes"
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,7,6,2.99,130,22.99,G,Deleted Scenes


In [17]:
# Chequeemos el resto de columnas:

film.film_id.value_counts() # todo correcto

1       1
672     1
659     1
660     1
661     1
       ..
339     1
340     1
341     1
342     1
1000    1
Name: film_id, Length: 1000, dtype: int64

In [18]:
film.title.value_counts() # también

ACADEMY DINOSAUR    1
PERFECT GROOVE      1
PARK CITIZEN        1
PARTY KNOCK         1
PAST SUICIDES       1
                   ..
FROGMEN BREAKING    1
FRONTIER CABIN      1
FROST HEAD          1
FUGITIVE MAGUIRE    1
ZORRO ARK           1
Name: title, Length: 1000, dtype: int64

In [19]:
film.description.value_counts() # también

A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies                   1
A Thrilling Yarn of a Dog And a Dog who must Build a Husband in A Baloon                                           1
A Taut Epistle of a Sumo Wrestler And a Girl who must Face a Husband in Ancient Japan                              1
A Fateful Display of a Technical Writer And a Butler who must Battle a Sumo Wrestler in An Abandoned Mine Shaft    1
A Intrepid Tale of a Madman And a Astronaut who must Challenge a Hunter in A Monastery                             1
                                                                                                                  ..
A Unbelieveable Yarn of a Mad Scientist And a Cat who must Chase a Lumberjack in Australia                         1
A Emotional Story of a Madman And a Waitress who must Battle a Teacher in An Abandoned Fun House                   1
A Amazing Reflection of a Lumberjack And a Cat who must Discover

In [20]:
film.release_year.value_counts() # Uy.... ¿TODAS las películas del 2006? Qué raro. Esto igual está mal.

2006    1000
Name: release_year, dtype: int64

Bueno. Después de una búsqueda general de algunas películas en internet, he concluido que quizá no sean películas reales. De momento no hay manera de comprobar si son todas del 2006 o no, pero es información importante para un videoclub así que la dejamos estar.

In [21]:
film.language_id.value_counts() # Todas tienen el mismo language id. 
# Esto no es disparatado y podrían entrar más de otros idiomas si el videoclip crece. La dejamos.

1    1000
Name: language_id, dtype: int64

In [22]:
film.rental_duration.value_counts() # Todo normal, duración en días del préstamo (estipulada? media?)

6    212
3    203
4    203
7    191
5    191
Name: rental_duration, dtype: int64

In [23]:
film.rental_rate.value_counts() # Todo normal, aunque no sé muy bien qué es este valor

0.99    341
4.99    336
2.99    323
Name: rental_rate, dtype: int64

In [24]:
film.length.unique() # Todo normal, duración en minutos en formato entero

array([ 86,  48,  50, 117, 130, 169,  62,  54, 114,  63, 126, 136, 150,
        94,  46, 180,  82,  57, 113,  79, 129,  85,  92, 181,  74, 179,
        91, 168, 119, 153, 147, 127, 121,  68,  99, 148, 137, 170,  83,
       108, 118, 162, 182,  75, 173,  87,  65,  90, 122, 160,  89, 175,
       106,  73, 151, 100,  53,  77, 142,  93, 163, 103,  61,  71, 102,
        76,  98,  72, 176, 123,  56, 161,  60, 133, 125,  67,  52, 120,
       167,  70, 135, 112, 183, 110, 152,  51, 146,  66, 185, 124, 107,
       101, 132, 143, 165,  81, 164,  58,  95, 139,  55, 149, 109,  59,
       172, 115, 184, 166,  64, 104,  69,  84, 144,  88, 141,  47,  49,
       177, 154, 159, 178,  96, 116, 171, 140, 155, 158, 174, 138,  97,
       131, 156,  80, 145, 111, 128, 157,  78, 105, 134])

In [25]:
film.replacement_cost.unique() # Todo bien, coste de reemplazo en formato float

array([20.99, 12.99, 18.99, 26.99, 22.99, 17.99, 28.99, 15.99, 21.99,
       24.99, 16.99, 23.99, 10.99, 14.99, 27.99,  9.99, 19.99, 11.99,
       29.99, 25.99, 13.99])

In [26]:
film.rating.unique() # Todo bien, es el Motion Picture Association film rating system

array(['PG', 'G', 'NC-17', 'PG-13', 'R'], dtype=object)

In [27]:
film.special_features.unique() # Interesante. Esta columna igual interesa dividira en más cachos

array(['Deleted Scenes,Behind the Scenes', 'Trailers,Deleted Scenes',
       'Commentaries,Behind the Scenes', 'Deleted Scenes', 'Trailers',
       'Commentaries,Deleted Scenes',
       'Trailers,Deleted Scenes,Behind the Scenes',
       'Trailers,Commentaries,Behind the Scenes', 'Trailers,Commentaries',
       'Trailers,Behind the Scenes',
       'Commentaries,Deleted Scenes,Behind the Scenes',
       'Trailers,Commentaries,Deleted Scenes',
       'Trailers,Commentaries,Deleted Scenes,Behind the Scenes',
       'Behind the Scenes', 'Commentaries'], dtype=object)

In [28]:
# Vamos a ello.

film['deleted_scenes'] = [1 if 'deleted' in i.lower() else 0 for i in film.special_features]
film['behind_scenes'] = [1 if 'behind' in i.lower() else 0 for i in film.special_features]
film['trailer'] = [1 if 'trailer' in i.lower() else 0 for i in film.special_features]
film['commentaries'] = [1 if 'commentaries' in i.lower() else 0 for i in film.special_features]

# Eliminamos la otra columna: 

film.drop(columns = 'special_features', inplace = True)

film.head()


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,deleted_scenes,behind_scenes,trailer,commentaries
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,7,6,0.99,86,20.99,PG,1,1,0,0
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,7,3,4.99,48,12.99,G,1,0,1,0
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,7,7,2.99,50,18.99,NC-17,1,0,1,0
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,7,5,2.99,117,26.99,G,0,1,0,1
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,7,6,2.99,130,22.99,G,1,0,0,0


In [29]:
# Tabla limpia. A por la conexión entre actores y películas.

# Tabla actorfilm

In [30]:
firstglance(actorfilm)

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14



Shape: (1000, 5)

Null values count:

first_name      0
last_name       0
title           0
release_year    0
category_id     0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


None

In [31]:
# Tiene 1000 filas, igual que el número de películas, por lo que estará agrupada en torno a esta categoría. 
# Unimos nombres y apellidos como hicimos antes:

actorfilm['name'] = actorfilm.apply(lambda x:'%s_%s' % (x['first_name'],x['last_name']), axis = 1)

# Nos deshacemos de las dos antiguas:

actorfilm.drop(columns = ['first_name','last_name'], inplace = True)

actorfilm.head()

,title,release_year,category_id,name
0,ACADEMY DINOSAUR,2006,6,PENELOPE_GUINESS
1,ANACONDA CONFESSIONS,2006,2,PENELOPE_GUINESS
2,ANGELS LIFE,2006,13,PENELOPE_GUINESS
3,BULWORTH COMMANDMENTS,2006,10,PENELOPE_GUINESS
4,CHEAPER CLYDE,2006,14,PENELOPE_GUINESS


In [32]:
# Como esta será la tabla de conexión entre autores y películas, vamos a cambiar el nombre y el título 
# por el id correspondiente en cada caso que será más eficiente.
# Creamos diccionarios para emparejar cada película con su film_id, y cada actor con su actor_id:

film_dict = dict(zip(film.title.values, film.film_id.values))
actor_dict = dict(zip(actor.name.values, actor.actor_id.values))

film_dict

{'ACADEMY DINOSAUR': 1,
 'ACE GOLDFINGER': 2,
 'ADAPTATION HOLES': 3,
 'AFFAIR PREJUDICE': 4,
 'AFRICAN EGG': 5,
 'AGENT TRUMAN': 6,
 'AIRPLANE SIERRA': 7,
 'AIRPORT POLLOCK': 8,
 'ALABAMA DEVIL': 9,
 'ALADDIN CALENDAR': 10,
 'ALAMO VIDEOTAPE': 11,
 'ALASKA PHANTOM': 12,
 'ALI FOREVER': 13,
 'ALICE FANTASIA': 14,
 'ALIEN CENTER': 15,
 'ALLEY EVOLUTION': 16,
 'ALONE TRIP': 17,
 'ALTER VICTORY': 18,
 'AMADEUS HOLY': 19,
 'AMELIE HELLFIGHTERS': 20,
 'AMERICAN CIRCUS': 21,
 'AMISTAD MIDSUMMER': 22,
 'ANACONDA CONFESSIONS': 23,
 'ANALYZE HOOSIERS': 24,
 'ANGELS LIFE': 25,
 'ANNIE IDENTITY': 26,
 'ANONYMOUS HUMAN': 27,
 'ANTHEM LUKE': 28,
 'ANTITRUST TOMATOES': 29,
 'ANYTHING SAVANNAH': 30,
 'APACHE DIVINE': 31,
 'APOCALYPSE FLAMINGOS': 32,
 'APOLLO TEEN': 33,
 'ARABIA DOGMA': 34,
 'ARACHNOPHOBIA ROLLERCOASTER': 35,
 'ARGONAUTS TOWN': 36,
 'ARIZONA BANG': 37,
 'ARK RIDGEMONT': 38,
 'ARMAGEDDON LOST': 39,
 'ARMY FLINTSTONES': 40,
 'ARSENIC INDEPENDENCE': 41,
 'ARTIST COLDBLOODED': 42,
 'ATLAN

In [33]:
actor_dict

{'PENELOPE_GUINESS': 1,
 'NICK_WAHLBERG': 2,
 'ED_CHASE': 3,
 'JENNIFER_DAVIS': 4,
 'JOHNNY_LOLLOBRIGIDA': 5,
 'BETTE_NICHOLSON': 6,
 'GRACE_MOSTEL': 7,
 'MATTHEW_JOHANSSON': 8,
 'JOE_SWANK': 9,
 'CHRISTIAN_GABLE': 10,
 'ZERO_CAGE': 11,
 'KARL_BERRY': 12,
 'UMA_WOOD': 13,
 'VIVIEN_BERGEN': 14,
 'CUBA_OLIVIER': 15,
 'FRED_COSTNER': 16,
 'HELEN_VOIGHT': 17,
 'DAN_TORN': 18,
 'BOB_FAWCETT': 19,
 'LUCILLE_TRACY': 20,
 'KIRSTEN_PALTROW': 21,
 'ELVIS_MARX': 22,
 'SANDRA_KILMER': 23,
 'CAMERON_STREEP': 24,
 'KEVIN_BLOOM': 25,
 'RIP_CRAWFORD': 26,
 'JULIA_MCQUEEN': 27,
 'WOODY_HOFFMAN': 28,
 'ALEC_WAYNE': 29,
 'SANDRA_PECK': 30,
 'SISSY_SOBIESKI': 31,
 'TIM_HACKMAN': 32,
 'MILLA_PECK': 33,
 'AUDREY_OLIVIER': 34,
 'JUDY_DEAN': 35,
 'BURT_DUKAKIS': 36,
 'VAL_BOLGER': 37,
 'TOM_MCKELLEN': 38,
 'GOLDIE_BRODY': 39,
 'JOHNNY_CAGE': 40,
 'JODIE_DEGENERES': 41,
 'TOM_MIRANDA': 42,
 'KIRK_JOVOVICH': 43,
 'NICK_STALLONE': 44,
 'REESE_KILMER': 45,
 'PARKER_GOLDBERG': 46,
 'JULIA_BARRYMORE': 47,
 'FRANCES

In [34]:
# Usamos estos diccionarios para rellenar las columnas:

actorfilm['film_id'] = [film_dict[i] for i in actorfilm.title]
actorfilm['actor_id'] = [actor_dict[i] for i in actorfilm.name]

actorfilm.head()

# Contamos cuántos actores de la tabla 'actor' no están en la columna mixta, y lo mismo con las 
# películas de 'film': 

c1 = film.shape[0] - actorfilm.film_id.unique().shape[0]
c2 = actor.shape[0] - actorfilm.actor_id.unique().shape[0]


display(actorfilm)

print('\nHay',c1,'películas que no aparecen en la lista de conexión actorfilm, y',c2,'actores que tampoco aparecen.')

,title,release_year,category_id,name,film_id,actor_id
0,ACADEMY DINOSAUR,2006,6,PENELOPE_GUINESS,1,1
1,ANACONDA CONFESSIONS,2006,2,PENELOPE_GUINESS,23,1
2,ANGELS LIFE,2006,13,PENELOPE_GUINESS,25,1
3,BULWORTH COMMANDMENTS,2006,10,PENELOPE_GUINESS,106,1
4,CHEAPER CLYDE,2006,14,PENELOPE_GUINESS,140,1
...,...,...,...,...,...,...
995,COMANCHEROS ENEMY,2006,3,GOLDIE_BRODY,168,39
996,DAISY MENAGERIE,2006,14,GOLDIE_BRODY,203,39
997,DESERT POSEIDON,2006,11,GOLDIE_BRODY,222,39
998,EVERYONE CRAFT,2006,9,GOLDIE_BRODY,290,39



Hay 386 películas que no aparecen en la lista de conexión actorfilm, y 160 actores que tampoco aparecen.


Parece que hay muchos actores de la tabla 'actor' y muchas películas de la tabla 'film' que no están relacionados a través de la tabla de conexión. Quizá sea porque el dueño del videoclub no las tiene en el inventario, pero de cualquier manera no nos estorban. Lo importante es tener bien etiquetadas las conexiones entre ambas tablas en lo posible.. deshagámonos de 'title', 'name' y 'release_year' (esta última ya está en la tabla 'film').

In [35]:
actorfilm.drop(columns = ['release_year','name','title'], inplace = True)

actorfilm # Ale, tabla limpita

,category_id,film_id,actor_id
0,6,1,1
1,2,23,1
2,13,25,1
3,10,106,1
4,14,140,1
...,...,...,...
995,3,168,39
996,14,203,39
997,11,222,39
998,9,290,39


# Tabla language

In [36]:
firstglance(language)

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19



Shape: (6, 3)

Null values count:

language_id    0
name           0
last_update    0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


None

Vaya tabla más enana. Parece que todas las películas que tenemos están disponibles en inglés (id = 1 en la tabla film), pero está bien no haber quitado esa columna porque según los datos parece que nuestro emprendedor tiene intención de tener películas en más idiomas en un futuro.

Guardamos el last_update fuera, lo metemos en la tabla de updates y a correr.

In [37]:
a = language.last_update[0]
updates = pd.concat([updates, pd.DataFrame({'table': 'language','last_update':[a]})], axis = 0)
language.drop(columns = 'last_update', inplace = True)

# Cambiemos el nombre de las columnas:
language.columns = ['id','lan']

# Vamos a añadir el idioma desconocido con id 7 que mencionamos antes:
language = pd.concat([language, pd.DataFrame({'id': 7,'lan':['Unknown']})], axis = 0)

language # ¡listo!

,id,lan
0,1,English
1,2,Italian
2,3,Japanese
3,4,Mandarin
4,5,French
5,6,German
0,7,Unknown


# Tabla rental

In [38]:
# Vamos a ver qué tiene dentro:

firstglance(rental)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53



Shape: (1000, 7)

Null values count:

rental_id       0
rental_date     0
inventory_id    0
customer_id     0
return_date     0
staff_id        0
last_update     0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
 6   last_update   1000 non-null   object
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


None

Interesante. Parece que había clientes identificados, staff y demás, pero no hay una tabla externa con ellos. Sería útil tener una de cada, aunque fuera sólo con un único registro (no tiene sentido dejar los ids actuales si no tenemos nombres ni relaciones más allá), y dejarlas abiertas para que el dueño del negocio las use.
Primero hacemos lo propio con la columna de last_update y luego vamos a ello.

In [39]:
# Comprobamos que es un valor único:

rental.last_update.value_counts() # Comprobado. Columna fuera:

a = rental.last_update[0]
updates = pd.concat([updates, pd.DataFrame({'table': 'rental','last_update':[a]})], axis = 0)
rental.drop(columns = 'last_update', inplace = True)

rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1


In [40]:
updates # Echemos un ojo de paso a cómo está nuestra tabla de updates

,table,last_update
0,actor,2006-02-15 04:34:33
0,film,2006-02-15 05:03:42
0,language,2006-02-15 05:02:19
0,rental,2006-02-15 21:30:53


Va bien, reiniciaremos el índice al final. Vamos a eliminar las columnas de staff_id y de customer_id,
y crearemos sendos dataframes vacíos (o bueno, con un único registro), para dejar la estructura de esas 
tablas hechas.

In [41]:
customers = pd.DataFrame({'cust_id':[1],'name':['JASON_BLUE'],'sex':['female'],'age':[29],'phone':['+34 675879554'],
                          'mail':'jbluecoulds@gmail.com','postal':[28766]}) 

# Metemos algunos campos que pueden ser útiles de cara a estudios estadísticos futuros, como el código postal, la
# edad o el género.

staff = pd.DataFrame({'staff_id':[1],'name':['JASON_BLUE'],'phone':['+34 675879554'],
                      'mail':'jbluecoulds@gmail.com'})

# En principio no vamos a dejar opción a que haya varias tiendas, así que no pondremos una columna de store_id 
# en staff.

display(customers)
display(staff)

,cust_id,name,sex,age,phone,mail,postal
0,1,JASON_BLUE,female,29,+34 675879554,jbluecoulds@gmail.com,28766


,staff_id,name,phone,mail
0,1,JASON_BLUE,+34 675879554,jbluecoulds@gmail.com


Con respecto a los datos de clientes antiguos, es interesante conservarlos de cara a futuros estudios de mercado posibles. Sin embargo, el propósito de este proyecto es crear la estructura de la database, por lo que borraremos la información antigua y dejaremos un solo registro para tener la tabla de rental inicializada. 

Le diremos al empresario que se guarde el csv de datos originales para un futuro, y a correr. 
Además, añadiremos una columna con la duración del préstamo, y otra con el film_id para saber qué película ha alquilado. 

De esta manera será fácil desde la base de datos de SQL saber si el cliene ha respetado el límite de tiempo establecido en la tabla film o no.

In [42]:
rental.drop(index = list(range(1,rental.shape[0])), inplace = True)

rental['film_id'] = 1

# Primero convertimos las columnas de return_date y rental_date a formato fecha, que ahora mismo son strings:

# pd.to_datetime(pd.Series("2020-12-08"))
# change_format = date_sr.dt.strftime('%d/%m/%Y')

rental.return_date = pd.to_datetime(rental.return_date)
rental.rental_date = pd.to_datetime(rental.rental_date)
rental['rental_time'] = (rental.return_date - rental.rental_date).dt.days 

# Para la diferencia nos quedamos solo con los días enteros, que son los que interesan de cara al límite temporal
# de préstamo. Aún así nos quedamos con la fecha completa de alquiler y entrega incluyendo horas en las columnas
# originales.

# Reordenamos un poco las columnas y listo.

rental = rental.reindex(columns = ['rental_id','customer_id','film_id','inventory_id',
                                   'staff_id','rental_time','rental_date','return_date'])
rental.customer_id = 1
rental.inventory_id = 1

rental.head() # Hecho

,rental_id,customer_id,film_id,inventory_id,staff_id,rental_time,rental_date,return_date
0,1,1,1,1,1,1,2005-05-24 22:53:30,2005-05-26 22:04:30


# Tabla Inventory

In [43]:
firstglance(inventory)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17



Shape: (1000, 4)

Null values count:

inventory_id    0
film_id         0
store_id        0
last_update     0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


None

In [44]:
# Empecemos por guardar el last_update en su sitio correspondiente:

a = inventory.last_update[0]
updates = pd.concat([updates, pd.DataFrame({'table': 'inventory','last_update':[a]})], axis = 0)
inventory.drop(columns = 'last_update', inplace = True)

inventory.head()

,inventory_id,film_id,store_id
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,2


In [45]:
# Esta tabla tiene información sobre varias tiendas. Echemos un ojo a los value counts.

inventory.store_id.value_counts() 

1    503
2    497
Name: store_id, dtype: int64

In [46]:
# Son dos... pero ya hemos dicho que de dos tiendas nada, que empezamos por una y ya vemos.
# Vamos a cargarnos esta columna y a ver cuántas pelis había en el inventario y cuántas repetidas:

inventory.drop(columns = 'store_id', inplace = True)

In [47]:
sum(inventory.inventory_id.unique() != range(1,1001))

0

In [48]:
# Vale, parece que había 1000 películas etiquetadas en el inventario en que ningún inventory_id se repite,
# van del 1 al 1000 sin huecos.

# Vamos a ver qué film_id había:

inventory.film_id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  34,  35,  37,  39,  40,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
       125, 126, 127, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 145, 146, 147, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 19

In [49]:
# Vemos que hay algunos huecos, como por ejemplo el film_id 14 que no está o el 36. 
# Echemos un ojo al total de pelis distintas que había:

len(inventory.film_id.unique())

207

In [50]:
# Había 207 películas identificadas inventariadas. Vamos a ver cuántas de estas están en la tabla de actorfilm..

A = set(inventory.film_id.unique())
B = set(actorfilm.film_id.unique())

len(A.intersection(B))

135

In [51]:
# Vaya. Pues no todas las películas que había disponibles están registradas en actorfilm... 
# Se ve que hay cosillas que arreglar, pero no podemos hacerlo sin más información. 
# Dejémoslo estar, le diremos al dueño del negocio que tiene que completar algunas tablas y listo.

inventory.head() # Tabla hecha!

,inventory_id,film_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


# Tabla category

In [52]:
firstglance(category)

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27



Shape: (16, 3)

Null values count:

category_id    0
name           0
last_update    0
dtype: int64

Table info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category_id  16 non-null     int64 
 1   name         16 non-null     object
 2   last_update  16 non-null     object
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


None

In [53]:
# Echemos un ojo a todas las categorías:

category

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27
5,6,Documentary,2006-02-15 04:46:27
6,7,Drama,2006-02-15 04:46:27
7,8,Family,2006-02-15 04:46:27
8,9,Foreign,2006-02-15 04:46:27
9,10,Games,2006-02-15 04:46:27


In [54]:
# Todo normal. Guardemos el last_update en su sitio:

a = category.last_update[0]
updates = pd.concat([updates, pd.DataFrame({'table': 'category','last_update':[a]})], axis = 0)
category.drop(columns = 'last_update', inplace = True)

# Cambiemos el nombre de la columna 'name' por 'cat', para que sea más claro:

category.columns = ['id','cat']

# Añadimos un id al final, el 17, para películas sin categorizar aún:

category = pd.concat([category, pd.DataFrame({'id': 17,'cat':['Unknown']})], axis = 0)


category.head() # Tabla lista

,id,cat
0,1,Action
1,2,Animation
2,3,Children
3,4,Classics
4,5,Comedy


# Tabla updates

In [55]:
# Vamos a añadir filas en updates para las tablas nuevas o las tablas que no han sido actualizadas en general:

updates = pd.concat([updates, pd.DataFrame({'table': 'staff','last_update':[pd.Timestamp.now()]})], axis = 0)
updates = pd.concat([updates, pd.DataFrame({'table': 'customers','last_update':[pd.Timestamp.now()]})], axis = 0)
updates = pd.concat([updates, pd.DataFrame({'table': 'actorfilm','last_update':[pd.Timestamp.now()]})], axis = 0)


# Y pasaremos el formato de las entradas a datetime:

updates.last_update = pd.to_datetime(updates.last_update)

updates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 0 to 0
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   table        9 non-null      object        
 1   last_update  9 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 216.0+ bytes


In [56]:
# Y reiniciamos el índice:

updates.reset_index(inplace = True, drop = True)

updates # Hecho!

,table,last_update
0,actor,2006-02-15 04:34:33.000000
1,film,2006-02-15 05:03:42.000000
2,language,2006-02-15 05:02:19.000000
3,rental,2006-02-15 21:30:53.000000
4,inventory,2006-02-15 05:09:17.000000
5,category,2006-02-15 04:46:27.000000
6,staff,2023-07-31 02:46:16.822367
7,customers,2023-07-31 02:46:16.823556
8,actorfilm,2023-07-31 02:46:16.824501


Como último retoque, estaría bien que el género de la película (category_id) no estuviera en actorfilm, sino en la propia tabla de películas. Vamos a hacer un traslado de una a otra a través de un diccionario como antes.

In [57]:
cat_dict = dict(zip(actorfilm.film_id,actorfilm.category_id))

cat_dict

{1: 6,
 23: 2,
 25: 13,
 106: 10,
 140: 14,
 166: 4,
 277: 11,
 361: 15,
 438: 10,
 499: 8,
 506: 11,
 509: 3,
 605: 9,
 635: 13,
 749: 11,
 832: 8,
 939: 5,
 970: 4,
 980: 12,
 3: 6,
 31: 8,
 47: 9,
 105: 1,
 132: 14,
 145: 8,
 226: 13,
 249: 4,
 314: 2,
 321: 13,
 357: 4,
 369: 14,
 399: 9,
 458: 8,
 481: 13,
 485: 3,
 518: 16,
 540: 12,
 550: 8,
 555: 5,
 561: 13,
 742: 10,
 754: 5,
 811: 16,
 958: 7,
 17: 12,
 40: 6,
 42: 15,
 87: 16,
 111: 1,
 185: 9,
 289: 13,
 329: 1,
 336: 6,
 341: 4,
 393: 6,
 441: 6,
 453: 15,
 480: 4,
 539: 7,
 618: 7,
 685: 13,
 827: 7,
 966: 6,
 967: 14,
 971: 9,
 996: 6,
 56: 1,
 62: 6,
 79: 7,
 355: 2,
 379: 7,
 398: 12,
 463: 15,
 490: 13,
 616: 6,
 691: 15,
 712: 6,
 714: 14,
 721: 12,
 798: 14,
 858: 5,
 909: 11,
 924: 14,
 19: 1,
 54: 12,
 85: 6,
 146: 7,
 171: 11,
 172: 7,
 202: 5,
 203: 14,
 286: 16,
 288: 16,
 316: 10,
 340: 13,
 375: 1,
 383: 15,
 392: 3,
 411: 12,
 503: 15,
 535: 11,
 571: 6,
 650: 6,
 665: 11,
 687: 6,
 730: 13,
 732: 1,
 817: 

In [58]:
# Ahora nos vamos a la tabla de film y construimos la nueva columna.

# Esta función nos ayudará a establecer un valor de 17 para aquellas películas del registro que
# no pertenezcan a una categoría concreta ('Unknown'):

def exchange(i): 
    try: return cat_dict[i]
    except: return 17

film['category_id'] = [exchange(i) for i in film.film_id]

film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,deleted_scenes,behind_scenes,trailer,commentaries,category_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,7,6,0.99,86,20.99,PG,1,1,0,0,6
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,7,3,4.99,48,12.99,G,1,0,1,0,11
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,7,7,2.99,50,18.99,NC-17,1,0,1,0,6
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,7,5,2.99,117,26.99,G,0,1,0,1,17
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,7,6,2.99,130,22.99,G,1,0,0,0,17


In [59]:
# Eliminamos la columna de category_id de actorfilm y listo:

actorfilm.drop(columns = 'category_id', inplace = True)

actorfilm.head() # Hecho

,film_id,actor_id
0,1,1
1,23,1
2,25,1
3,106,1
4,140,1


# Guardado de datos

Ahora que los datos están limpios, los guardamos en nuevos csv. En el siguiente notebook acabaremos de organizarlos ya teniendo más en mente el ERD final de la base de datos que crearemos en SQL.

In [60]:
# Guardamos los datos de todas las tablas

actor.to_csv('../data/actor_clean.csv', index = False)
category.to_csv('../data/category_clean.csv', index = False)
film.to_csv('../data/film_clean.csv', index = False)
language.to_csv('../data/language_clean.csv', index = False) 
actorfilm.to_csv('../data/actorfilm_clean.csv', index = False)
inventory.to_csv('../data/inventory_clean.csv', index = False)
updates.to_csv('../data/updates_clean.csv', index = False)
staff.to_csv('../data/staff_clean.csv', index = False)
customers.to_csv('../data/customers_clean.csv', index = False)
rental.to_csv('../data/rental_clean.csv', index = False)